[![Fixel Algorithms](https://fixelalgorithms.co/images/CCExt.png)](https://fixelalgorithms.gitlab.io)

# Optimization Methods

## Convex Optimization - Smooth Optimization - Localization by DF (Direction Finding) 

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 1.0.000 | 12/05/2025 | Royi Avital | First version                                                      |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/AIProgram/2024_02/0002PointLine.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

# Miscellaneous
from platform import python_version
import random

# Visualization
import matplotlib.pyplot as plt

# Jupyter
from IPython import get_ipython

## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

Code Notations:

```python
someVar    = 2; #<! Notation for a variable
vVector    = np.random.rand(4) #<! Notation for 1D array
mMatrix    = np.random.rand(4, 3) #<! Notation for 2D array
tTensor    = np.random.rand(4, 3, 2, 3) #<! Notation for nD array (Tensor)
tuTuple    = (1, 2, 3) #<! Notation for a tuple
lList      = [1, 2, 3] #<! Notation for a list
dDict      = {1: 3, 2: 2, 3: 1} #<! Notation for a dictionary
oObj       = MyClass() #<! Notation for an object
dfData     = pd.DataFrame() #<! Notation for a data frame
dsData     = pd.Series() #<! Notation for a series
hObj       = plt.Axes() #<! Notation for an object / handler / function handler
```

### Code Exercise

 - Single line fill

```python
valToFill = ???
```

 - Multi Line to Fill (At least one)

 ```python
 # You need to start writing
 ?????
 ```

 - Section to Fill

```python
#===========================Fill This===========================#
# 1. Explanation about what to do.
# !! Remarks to follow / take under consideration.
mX = ???

?????
#===============================================================#
```

In [ ]:
# Configuration
%matplotlib inline

# warnings.filterwarnings("ignore")

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# Matplotlib default color palette
lMatPltLibclr = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
# sns.set_theme() #>! Apply SeaBorn theme
# sns.set_palette("tab10")

runInGoogleColab = 'google.colab' in str(get_ipython())

In [ ]:
# Constants

FIG_SIZE_DEF    = (8, 8)
ELM_SIZE_DEF    = 50
CLASS_COLOR     = ('b', 'r')
EDGE_COLOR      = 'k'
MARKER_SIZE_DEF = 10
LINE_WIDTH_DEF  = 2

In [ ]:
# Course Packages


In [ ]:
# Auxiliary Functions


## Direction Finding Localization

DF ([Direction Finding](https://en.wikipedia.org/wiki/Direction_finding)) is a simple method to localize the location of an RF transmitter.

In classical DF for localization of RF Signal, each measurement yield a direction.  
When the direction is plotted on the map it creates a single line where the transmitter is believed to be on.  
With perfect measurements the intersection of 2 lines will reveal the location of the transmitter.  

Yet in practice, due to different inaccuracies in the model and measurements the real location is not at the intersection.  
One way to overcome some of those "noises" is use more than 2 measurements for the estimation.

![](https://i.imgur.com/HDt0YoF.png) Image Credit: [School Amateur Radio Club Network - Amateur Radio Direction Finding](https://www.sarcnet.org/amateur-radio-direction-finding.html)
<!-- ![](https://i.postimg.cc/8PsJwWYj/image.png) -->

* <font color='brown'>(**#**)</font> Using intersection of lines for localization is called [Triangulation](https://en.wikipedia.org/wiki/Triangulation).
* <font color='brown'>(**#**)</font> Farther reading on DF:
   - [Paul Denisowski - An Introduction to Radio Direction Finding Methodologie](http://www.denisowski.org/Publications/Denisowski%20-%20An%20Introduction%20to%20Radio%20Direction%20Finding%20Methodologies.pdf).
   - [Rohde Schwarz - An Introduction to Direction Finding Methodologies (White Paper)](https://cdn.rohde-schwarz.com/am/us/campaigns_2/a_d/Intro-to-direction-finding-methodologies~1.pdf).

### The Sum of Distances Minimization  

One approach to solve it is to find the point which minimizes the sum of distances to the lines:


$$ \arg \min_{\boldsymbol{x} \in \mathbb{R}^{2}} \sum_{i = 1}^{n} d \left( \boldsymbol{x}, \boldsymbol{l}_{i} \right) $$

Where $\boldsymbol{l}_{i}$ is the set of parameters of the $i$ -th line.  

This notebooks solve the problem for the case the distance is defined as the **Squared Euclidean** Distance.  
Given a set of lines, it derives and finds the point which minimizes the sum of squared distances.

* <font color='brown'>(**#**)</font> The accuracy of the model is mostly determined by the distance / dissimilarity function in use.  
In most cases the Squared Euclidean Distance is not a good measure to find the actual location.  
* <font color='brown'>(**#**)</font> The Squared Euclidean Distance does not form a valid metric as it does not hold the _Triangle Inequality_.  
  For instance: $ {\left| -2 - 2 \right|}^{2} \nleq {\left| -2 - 0 \right|}^{2} + {\left| 0 - 2 \right|}^{2}$.
* <font color='red'>(**?**)</font> Will the solution ofr the Squared Euclidean Distance be the same as the Euclidean Distance?

In [ ]:
# Parameters

tuGrid = (-2, 2, 1000) #<! Left boundary, right boundary, num points

## Generate Data


### Create 1D Grid

In [ ]:
# Generate 1D Grid

vX = np.linspace(tuGrid[0], tuGrid[1], tuGrid[2]) #<! Linear steps grid

print(f'The Grid Shape: {vX.shape}')

### Create Lines

Generate the `numLines` lines in 2 forms:

1. $a x + b y + c = 0$.
2. $y = m x + n$.

* <font color='brown'>(**#**)</font> The forms are equivalent. Namely the lines are the same, just in 2 different forms.

In [ ]:
# Generate the Lines
# Lines parameters: a x + b y + c = 0

mL = np.array([[1, 1, 0], [-0.8, 1.7, -0.7], [-0.8, 1.6, -0.75], [1.1, -0.95, -0.05]]) #<! Lines parameters: a, b, c
numLines = mL.shape[0]

#===========================Fill This===========================#
# 1. Normalize the lines so that a^2 + b^2 = 1.
# !! Make sure the line is the same.

vN = np.linalg.norm(mL[:, :2], axis = 1) #<! Calculate the norm of (a, b)
mL /= vN[:, None]                        #<! Scales all parameters

#===============================================================#

In [ ]:
# Convert the Lines into the y = m x + n Form

# The motivation is for visualization
mM = np.column_stack((-mL[:, 0] / mL[:, 1], -mL[:, 2] / mL[:, 1]))

In [ ]:
# Draw the Data

hF, hA = plt.subplots(figsize = (10, 7))

for ii in range(numLines):
    vY = np.array([mM[ii, 0] * valX + mM[ii, 1] for valX in vX])
    hA.plot(vX, vY, lw = LINE_WIDTH_DEF, label = f'Line {ii + 1:02d}')

hA.set_xlabel('x')
hA.set_ylabel('y')
hA.set_title('Set of Lines')

hA.legend();

## Deriving the Objective Function

This section formulates the problem as a _Quadratic Programming_ problem.

### Distance of a Point from a Line

Given a point $\boldsymbol{x} = \left[ {x}_{0}, {y}_{0} \right]^{T}$ then the _Euclidean Distance_ to a line, given by the parameters $l = \left( a, b, c \right) : a x + b y + c = 0$ is given by:

$$ d \left( \boldsymbol{x}, l \right) = \frac{ \left| a {x}_{0} + b {y}_{0} + c \right| }{ \sqrt{ {a}^{2} + {b}^{2} } } $$

Assuming all lines are normalized, then the operation becomes:

$$ d \left( \boldsymbol{x}, l \right) = a {x}_{0} + b {y}_{0} + c $$

The objective function, based on the _Squared Euclidean Distance_, for the DF as defined is given by:

$$ \arg \min_{\boldsymbol{x} \in \mathbb{R}^{2}} \sum_{i = 1}^{n} {d}^{2} \left( \boldsymbol{x}, \boldsymbol{l}_{i} \right) $$


* <font color='brown'>(**#**)</font> Using the Squared Euclidean as a measure of distance greatly simplifies the derivation of the solution.

### Question 1

Simplify the objective function to a _Quadratic Form_.

Guidance:
 - The Signed Distance as Inner Product  
   Define $\boldsymbol{w}_{i} = {\left[ {a}_{i}, {b}_{i} \right]}^{T}$.
 - Reformulate the Distance Function (The Squread Euclidean "Distance")  
   Write the distance function with the point $\boldsymbol{x}$ and the line.  
   Use inner product between $\boldsymbol{w}$ and $\boldsymbol{x}$.
 - Reformulate the Objective Function  
   Formulate the objective function as a _Quadratic Form_: $\boldsymbol{x}^{T} \boldsymbol{M} \boldsymbol{x} + 2 \boldsymbol{n}^{T} \boldsymbol{x} + o$.  
   Derive the values of $\boldsymbol{M}, \boldsymbol{n}, o$ with the line parameters.

</br>

* <font color='brown'>(**#**)</font> Quadratic Programming with no constraints is closely related to _Linear Least Squares_.

#### Answer 1

 - The distance as inner product: ${d}_{i}^{2} = {d}^{2} \left( \boldsymbol{x}, {l}_{i} \right) = {\left( \boldsymbol{w}_{i}^{T} \boldsymbol{x} + {c}_{i} \right)}^{2}$.
 - The distance function: ${d}_{i}^{2} = \boldsymbol{x}^{T} \boldsymbol{w}_{i} \boldsymbol{w}_{i}^{T} \boldsymbol{x} + 2 {c}_{i} \boldsymbol{w}_{i}^{T} \boldsymbol{x} + {c}_{i}^{2}$.
 - The objective function: $\sum {d}_{i}^{2} = \boldsymbol{x}^{T} \sum \left( \boldsymbol{w}_{i} \boldsymbol{w}_{i}^{T} \right) \boldsymbol{x} + 2 \sum \left( {c}_{i} \boldsymbol{w}_{i}^{T} \right) \boldsymbol{x} + \sum {c}_{i}^{2} = \boldsymbol{x}^{T} \boldsymbol{M} \boldsymbol{x} + 2 \boldsymbol{n}^{T} \boldsymbol{x} + o$

</br>

---

* <font color='red'>(**?**)</font> Is the problem _Convex_? Explain.

### Question 2

Derive the optimal solution for the problem.  
Compare it to the scalar case of parabolic function.

#### Answer 2

 - The function is defined by $f \left( \boldsymbol{x} \right) = \boldsymbol{x}^{T} \boldsymbol{M} \boldsymbol{x} + 2 \boldsymbol{n}^{T} \boldsymbol{x} + o$.
 - The gradient is given by: ${\nabla}_{x} f = 2 \boldsymbol{M} \boldsymbol{x} + 2 \boldsymbol{n}$.
 - The minimum, as the problem is _Convex_ is given at the vanishing point of the gradient: $\boldsymbol{x} = -\boldsymbol{M}^{-1} \boldsymbol{n}$.

</br>

---

* <font color='red'>(**?**)</font> Does $o$ have any affect on the solution? Should it be calculated?
* <font color='red'>(**?**)</font> Does ${c}_{i}$ affect the solution?

In [ ]:
# Find the Point to Minimize Sum of Squared Euclidean Distances

def PointMinSumSqrDistance( mL : np.ndarray ) -> np.ndarray:
    """
    The function finds the point in 2D which minimizes the sum of squared Euclidean distances to a set of lines.
    For a line, `vL`, a row in the matrix `mL`:
    The line is given as `vL[0] * x + vL[1] * y + vL[2] = 0`.  
    The line is assumed to have `np.linalg.norm(vL[:2]) = 1`.
    Input:
        mL      - Set of 2D lines parameters (numLines x 3).
    Output:
        vX      - 2D point which minimizes .
    """
    #===========================Fill This===========================#
    # 1. Calculate the matrix `mM`.
    # 2. Calculate the vector `vN`.
    # 3. Find the optimal point.
    # !! The solution should match the derivation.
    
    mM = mL[:, :2].T @ mL[:, :2]                         #<! Set mM
    vN = np.sum(mL[:, 2][:, None] * mL[:, :2], axis = 0) #<! Set vN

    vX = -sp.linalg.solve(mM, vN, assume_a = 'pos') #<! Solve the system of equations
    #===============================================================#
    
    return vX

* <font color='brown'>(**#**)</font> Sum of outer products: $\sum_{i} \boldsymbol{u}_{i} \boldsymbol{v}_{i}^{T} = \boldsymbol{U} \boldsymbol{V}^{T}$ where $\boldsymbol{U} = \begin{bmatrix} | & & | \\ \boldsymbol{u}_{1} & \dots & \boldsymbol{u}_{N} \\ | & & | \end{bmatrix}, \; \boldsymbol{V} = \begin{bmatrix} | & & | \\ \boldsymbol{v}_{1} & \dots & \boldsymbol{v}_{N} \\ | & & | \end{bmatrix}$.
* <font color='red'>(**?**)</font> Will the code `sp.linalg.solve(-mM, vN)` give the same result?
* <font color='red'>(**?**)</font> Will the code `sp.linalg.solve(mM, vN, assume_a = 'pos')` work?

In [ ]:
# Plot the Optimal Solution 

hF, hA = plt.subplots(figsize = (10, 7))

for ii in range(numLines):
    vY = np.array([mM[ii, 0] * valX + mM[ii, 1] for valX in vX])
    hA.plot(vX, vY, lw = LINE_WIDTH_DEF, label = f'Line {ii + 1:02d}')

vP = PointMinSumSqrDistance(mL) #<! Find the point
hA.scatter(vP[0], vP[1], s = 4 * MARKER_SIZE_DEF, color = '#FFE119', label = 'Optimal Point', zorder = 2.1)

hA.set_aspect('equal') #<! Must in order to have 90 [Deg]
hA.set_xlabel('x')
hA.set_ylabel('y')
hA.set_title('Set of Lines and the Optimal Point')

hA.legend();

* <font color='green'>(**@**)</font> Create _Heatmap_ of the objective function.
* <font color='green'>(**@**)</font> Create _Heatmap_ of the _Euclidean Distance_ and compare.
* <font color='red'>(**?**)</font> In what cases the Squared Euclidean Distance will fail?